# Processing Titles to Evaluate CM

In [1]:
%load_ext autoreload
%autoreload 2
%cd ~/SageMaker/merck_nl2sql/src/

/home/ec2-user/SageMaker/merck_nl2sql/src


In [45]:
import os
import config
from pipeline import nlq2SqlTool

In [3]:
question = 'Update on Clinical Aspects of Chronic Obstructive Pulmonary Disease.'

In [4]:
tool = nlq2SqlTool(config)
entities = tool.detect_entities(question)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [5]:
tool.process_entities(entities)

{'TIMEDAYS': [],
 'TIMEYEARS': [],
 'DRUG': [],
 'CONDITION': [{'BeginOffset': 30,
   'EndOffset': 67,
   'Text': 'Chronic Obstructive Pulmonary Disease',
   'Options': [{'Description': 'Chronic obstructive pulmonary disease, unspecified',
     'Code': 'J44.9',
     'Score': 0.8020230531692505},
    {'Description': 'Chronic obstructive pulmonary disease with (acute) exacerbation',
     'Code': 'J44.1',
     'Score': 0.7054653763771057},
    {'Description': 'Chronic obstructive pulmonary disease with acute lower respiratory infection',
     'Code': 'J44.0',
     'Score': 0.6053685545921326},
    {'Description': 'Chronic obstructive pyelonephritis',
     'Code': 'N11.1',
     'Score': 0.5842992067337036},
    {'Description': 'Pulmonary heart disease, unspecified',
     'Code': 'I27.9',
     'Score': 0.4796171188354492}],
   'Query-arg': 'J44.9',
   'Placeholder': '<ARG-CONDITION><0>'}],
 'AGE': [],
 'STATE': [],
 'GENDER': [],
 'ETHNICITY': [],
 'RACE': []}

In [42]:
titles_path = '/home/ec2-user/SageMaker/efs/data/pilot_nl2sql_dev/cm_evaluation/selected_titles_manual.txt'
output_path = '/home/ec2-user/SageMaker/efs/data/pilot_nl2sql_dev/cm_evaluation/selected_titles_manual_processed.txt'
with open(titles_path, 'r') as fp:
    titles = fp.readlines()
    titles = [title.strip() for title in titles]
print(len(titles))

229


In [60]:
def process_title(title0):
    """Detect entities in a given title and replace the detected ones with their concept codes."""
    output_title = title0
#     print(title0)
    detected = tool.process_entities(tool.detect_entities(title0))
    for ent_key, ent_value in detected.items():
        if ent_key =='DRUG' or ent_key == 'CONDITION':
            for value in ent_value:
                begin, end = value['BeginOffset'], value['EndOffset']
                text = value['Text']
                detections = value['Options']
                outputs = []
                for detection in detections:
#                     print(detection)
#                     print('+++++')
                    outputs.append(f"({detection['Description']}, {detection['Code']})")
                #print(text)
                outputs = f'<{ent_key},' + ' / '.join(outputs) + '>'
                output_title= output_title.replace(text, outputs)
    output = f"Input:\n{title0}\nOutput:\n{output_title}\n# of Correct:\n# of Errors:\n"
    return output

def process_titles(titles0):
    """Process all titles."""
    print('Processing titles through CM...')
    outputs = ''
    for title in titles0:
        output = process_title(title)
        outputs += output
    print('All titles processed successfully!')
    return outputs

def save_processed_titles(processed_titles, output_path):
    with open(output_path, 'w') as fp:
        fp.write(processed_titles)
    print('Processed data saved Successfully!')

In [61]:
#Process Titles by detecting entities and concept codes through CM
processed_titles = process_titles(titles)

Processing titles through CM...
All titles processed successfully!


In [62]:
#Save the processed data
save_processed_titles(processed_titles, output_path)

Processed data saved Successfully!
